In [1]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
import umap

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm

# Use IoU instead
# import re
from scipy.spatial.distance import cdist
from skimage.transform import resize
from time import time


d:\zsh\graduation\grad_env_take_2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the current df in

In [4]:
# original_path = "D:\\zsh\graduation\\Graduation-project-domain-shift-image-2-image\\noisy_data_sample_200.csv"
original_path = "D:\\zsh\graduation\\Graduation-project-domain-shift-image-2-image\\noise_data_sample_200_v2.csv"

In [5]:
original_df= pd.read_csv(original_path,index_col = "id")

In [6]:
original_df.tail()

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,umap_2,bottleneck_pca_embedding,bottleneck_tsne_embedding,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
id,,,,,,,,,,,,,,,,,,,,,
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample_new\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,4.354430,"[[0.520260214805603, -0.023931534960865974], [...","[[-12.483431816101074, -0.7725005149841309], [...",0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample_new\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,2.663829,"[[0.3765714466571808, -0.041820090264081955], ...","[[-4.8248372077941895, -16.330381393432617], [...",0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample_new\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,2.381734,"[[0.10845088958740234, 0.3767417371273041], [0...","[[-17.50540542602539, 6.707708358764648], [-7....",0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000
1198,198.jpeg,5,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample_new\198.jpeg,dataset\noise_data\predictions\5\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,...,3.467603,"[[0.8477775454521179, 0.13700152933597565], [0...","[[-19.99791145324707, 9.822242736816406], [-5....",0.405075,0.382986,0.000000,0.000000,0.244767,0.000000,0.108215
1199,199.jpeg,5,dataset\noise_data\images\5\199.jpeg,dataset\cityscapes_labels_sample_new\199.jpeg,dataset\noise_data\predictions\5\199.jpeg,0.184860,0.237762,0.115555,0.461823,0.000000,...,2.605118,"[[0.12233881652355194, -0.1914900541305542], [...","[[-8.874900817871094, -2.126358985900879], [-6...",0.369919,0.184443,0.437477,0.137569,0.192291,0.000000,0.000000


In [7]:
original_df.columns

Index(['name', 'noise_level', 'image_path', 'label_path', 'prediction_path',
       'other_ratio', 'road_ratio', 'sidewalk_ratio', 'vegetation_ratio',
       'sky_ratio', 'car_ratio', 'simple_tsne_1', 'simple_tsne_2',
       'meaningful_tsne_1', 'meaningful_tsne_2', 'pca_1', 'pca_2', 'umap_1',
       'umap_2', 'bottleneck_pca_embedding', 'bottleneck_tsne_embedding',
       'overall_iou', 'other_iou', 'road_iou', 'sidewalk_iou',
       'vegetation_iou', 'sky_iou', 'car_iou'],
      dtype='object')

Need to work on label path and prediction path

# Read the original masks

In [10]:
website_folder = "D:\\zsh\\graduation\\Graduation-project-domain-shift-image-2-image"
label_folder = os.path.join(website_folder,"dataset\\cityscapes_labels_sample")
# noise_data_path = "D:\\zsh\\graduation\\Graduation-project-domain-shift-image-2-image\\dataset\\noise_data"

In [29]:
new_label_folder = os.path.join(website_folder,"dataset\\cityscapes_labels_sample_new")

Labels

In [9]:
def find_index_2d(array, subarray):
    for index, row in enumerate(array):
        if np.array_equal(row, subarray):
            return index
#     print(subarray)
    return -1  # return -1 if the subarray is not found

In [10]:
class_names = ["others","road","sidewalk","vegetation","sky","car"]
colors = [[  0,   0,   0],
          [128, 64, 128],# road
          [244, 35, 232], # sidewalk
          [107, 142, 35],# vegetation
          [70, 130, 180], # sky
          [0, 0, 142], # car
         ]
# This function is particularly useful for noise domain
def color_to_class(old_mask):
    new_mask = np.zeros(shape=(old_mask.shape[0], old_mask.shape[1]), dtype = np.int32)
    for row in range(old_mask.shape[0]):
        for col in range(old_mask.shape[1]):
            class_pixel = old_mask[row,col]
#             class_ind = find_index_2d(colors,class_pixel)
            d = cdist(np.array([class_pixel]),np.array(colors))
            class_ind = np.argmin(d)
            new_mask[row,col] = class_ind
    return new_mask

In [11]:
# class to color: for viewing labels
selected_light_colors = {
    "others": [210, 157, 227],
    "road": [255, 196, 115],
    "sidewalk": [255, 255, 153],
    "vegetation": [230, 202, 145],
    "sky": [252, 167, 211],
    "car": [240, 130, 120]
}

def noise_class_to_color(labels):
    label_colors = np.zeros((256,256,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            # print(pixel, type(pixel))
            class_name = class_names[pixel]
            label_colors[i,j] = selected_light_colors[class_name]
    
    return label_colors.astype(np.uint8)  # make each pixel value an integer to visualize it better

In [52]:
new_label_paths =[] 
start_time = time()
for i in range(200):
    if i%40 ==0:
        print(i)
    label_name = str(i)+".jpeg"
    label_path = os.path.join(label_folder,label_name)
    old_label = Image.open(label_path).convert("RGB")
    new_label = color_to_class(np.array(old_label))
    
    new_label_image= noise_class_to_color(new_label)
    new_label_image = Image.fromarray(new_label_image.astype(np.uint8))
    
    new_full_path= os.path.join(new_label_folder,label_name)
    new_label_image.save(new_full_path)
    new_rel_path = os.path.relpath(new_full_path, website_folder)
    new_label_paths.append(new_rel_path)
#     end_time = time()
#     print(end_time-start_time)
#     break
#     if i>=9:
#         end_time = time()
#         print(end_time-start_time)
#         break

0
40
80
120
160


In [51]:
new_label_paths

['dataset\\cityscapes_labels_sample_new\\0.jpeg',
 'dataset\\cityscapes_labels_sample_new\\1.jpeg',
 'dataset\\cityscapes_labels_sample_new\\2.jpeg',
 'dataset\\cityscapes_labels_sample_new\\3.jpeg',
 'dataset\\cityscapes_labels_sample_new\\4.jpeg',
 'dataset\\cityscapes_labels_sample_new\\5.jpeg',
 'dataset\\cityscapes_labels_sample_new\\6.jpeg',
 'dataset\\cityscapes_labels_sample_new\\7.jpeg',
 'dataset\\cityscapes_labels_sample_new\\8.jpeg',
 'dataset\\cityscapes_labels_sample_new\\9.jpeg']

In [36]:
np.unique(np.array(new_label))

array([-1,  0,  4,  5])

In [33]:
print(end_time-start_time)

25.270159482955933


In [62]:
# load in all of the labels
new_df = original_df.copy()
new_df.drop(columns = ["label_path"])

,name,noise_level,image_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,car_ratio,...,umap_2,bottleneck_pca_embedding,bottleneck_tsne_embedding,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
id,,,,,,,,,,,,,,,,,,,,,
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\noise_data\predictions\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472,...,2.906299,"[[-0.013326136395335197, -0.30401840806007385]...","[[-8.655488967895508, 15.489809036254883], [-1...",0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\noise_data\predictions\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000,...,1.605131,"[[0.1603696197271347, -0.22683094441890717], [...","[[-14.934941291809082, 11.525232315063477], [-...",0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\noise_data\predictions\0\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748,...,5.091578,"[[0.019657980650663376, -0.3139321804046631], ...","[[-1.8404988050460815, 13.750861167907715], [-...",0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\noise_data\predictions\0\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498,...,4.119893,"[[0.1766330897808075, -0.19020713865756989], [...","[[-16.590892791748047, 5.171955108642578], [-0...",0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\noise_data\predictions\0\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937,...,3.517825,"[[0.924606204032898, 0.10404495894908905], [1....","[[-13.471735000610352, 9.674683570861816], [-1...",0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189,...,4.354430,"[[0.520260214805603, -0.023931534960865974], [...","[[-12.483431816101074, -0.7725005149841309], [...",0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884,...,2.663829,"[[0.3765714466571808, -0.041820090264081955], ...","[[-4.8248372077941895, -16.330381393432617], [...",0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357,...,2.381734,"[[0.10845088958740234, 0.3767417371273041], [0...","[[-17.50540542602539, 6.707708358764648], [-7....",0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000


In [56]:
new_label_paths_full = new_label_paths*6

In [59]:
new_label_paths_full[1055]

'dataset\\cityscapes_labels_sample_new\\55.jpeg'

In [ ]:
new_df["label_path"]=new_label_paths_full

In [64]:
new_df.to_csv("noise_data_sample_200_v2.csv",index = "id")

# Read the original prediction masks

In [8]:
website_folder = "D:\\zsh\\graduation\\Graduation-project-domain-shift-image-2-image"

noise_data_folder= os.path.join(website_folder,"dataset\\noise_data")
prediction_folder_old = os.path.join(noise_data_folder,"predictions")
prediction_folder_new = os.path.join(noise_data_folder,"predictions_new")

In [27]:
new_prediction_paths = []

In [40]:
# iterate over the different subfolder of prediction folder
for noise_level in os.listdir(prediction_folder_old):
    print(noise_level)
    noise_level_folder = os.path.join(prediction_folder_old,noise_level)
    for prediction_name in os.listdir(noise_level_folder):
        old_prediction_path = os.path.join(noise_level_folder,prediction_name)
        old_prediction = Image.open(old_prediction_path).convert("RGB")
        
        new_prediction = color_to_class(np.array(old_prediction))
    
        new_pred_image= noise_class_to_color(new_prediction)
        new_pred_image = Image.fromarray(new_pred_image.astype(np.uint8))
    
        new_full_path= os.path.join(prediction_folder_new,noise_level,prediction_name)
        new_pred_image.save(new_full_path)

0
1
2
3
4
5


In [41]:
# get all the names
new_prediction_paths = []
for noise_level in os.listdir(prediction_folder_old):
    print(noise_level)
    noise_level_folder = os.path.join(prediction_folder_old,noise_level)
    for i in range(200):
        prediction_name = str(i)+".jpeg"
        new_full_path= os.path.join(prediction_folder_new,noise_level,prediction_name)
        new_rel_path = os.path.relpath(new_full_path, website_folder)
        new_prediction_paths.append(new_rel_path)

0
1
2
3
4
5


In [42]:
# load in all of the labels
new_df = original_df.copy()
new_df.drop(columns = ["prediction_path"])
new_df["prediction_path"]=new_prediction_paths

In [43]:
new_df.to_csv("noise_data_sample_200_v3.csv",index = "id")

In [35]:
new_df

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,umap_2,bottleneck_pca_embedding,bottleneck_tsne_embedding,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
id,,,,,,,,,,,,,,,,,,,,,
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample_new\0.jpeg,dataset\noise_data\predictions_new\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,...,2.906299,"[[-0.013326136395335197, -0.30401840806007385]...","[[-8.655488967895508, 15.489809036254883], [-1...",0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample_new\1.jpeg,dataset\noise_data\predictions_new\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,...,1.605131,"[[0.1603696197271347, -0.22683094441890717], [...","[[-14.934941291809082, 11.525232315063477], [-...",0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample_new\2.jpeg,dataset\noise_data\predictions_new\0\10.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,...,5.091578,"[[0.019657980650663376, -0.3139321804046631], ...","[[-1.8404988050460815, 13.750861167907715], [-...",0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample_new\3.jpeg,dataset\noise_data\predictions_new\0\100.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,...,4.119893,"[[0.1766330897808075, -0.19020713865756989], [...","[[-16.590892791748047, 5.171955108642578], [-0...",0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample_new\4.jpeg,dataset\noise_data\predictions_new\0\101.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,...,3.517825,"[[0.924606204032898, 0.10404495894908905], [1....","[[-13.471735000610352, 9.674683570861816], [-1...",0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample_new\195.jpeg,dataset\noise_data\predictions_new\5\95.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,4.354430,"[[0.520260214805603, -0.023931534960865974], [...","[[-12.483431816101074, -0.7725005149841309], [...",0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample_new\196.jpeg,dataset\noise_data\predictions_new\5\96.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,2.663829,"[[0.3765714466571808, -0.041820090264081955], ...","[[-4.8248372077941895, -16.330381393432617], [...",0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample_new\197.jpeg,dataset\noise_data\predictions_new\5\97.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,2.381734,"[[0.10845088958740234, 0.3767417371273041], [0...","[[-17.50540542602539, 6.707708358764648], [-7....",0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000


# Important: from here is the original code!
(Relevant codes for discrete domain)

In [2]:
synthia_sample_number = 500 # or 500

In [3]:
# # the website folder, we make relative folder based on this as the source folder
# website_folder = "C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image"
# dir_path = "C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation"
# cityscapes_path = os.path.join(dir_path,"Cityscapes")
# synthia_path = os.path.join(dir_path,"Synthia")

# # note that each folder for synthia contains subfolders of citynames.
# cityscapes_image_folder = os.path.join(cityscapes_path,"leftImg8bit\\val")
# cityscapes_label_folder = os.path.join(cityscapes_path,"gtFine\\val") # also need to find the right labels
# cityscapes_predictions_folder = os.path.join(cityscapes_path,"outputs\\preds")

# if synthia_sample_number:
#     synthia_image_folder = os.path.join(synthia_path,"RGB_subset_")+str(synthia_sample_number) # all the images for now
# else:
#     synthia_image_folder = os.path.join(synthia_path,"RGB")

# # color mask
# synthia_label_image_folder = os.path.join(synthia_path,"GT\\COLOR") # also need to find the right labels
# # label class folder 
# synthia_label_class_folder = os.path.join(synthia_path,"GT\\LABELS")
# synthia_predictions_folder = os.path.join(synthia_path,"predictions_")+str(synthia_sample_number)+"\\predictions"

In [11]:
# root of the images
root_image_folder = cityscapes_image_folder
root_label_folder = cityscapes_label_folder
root_predictions_folder = cityscapes_predictions_folder
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

cityscapes_full_paths = []
# cityscapes_names = []
# cityscapes_rel_paths = []
cityscapes_initial_info = []
cityscapes_images = []
cityscapes_labels = []
cityscapes_predictions = []

# Walk through root folder and subfolders
for subdir, dirs, files in os.walk(root_image_folder):
    for file in files:
        # Check if the file has one of the desired extensions
        if any(file.lower().endswith(ext) for ext in image_extensions):
            # path of image
            full_image_path = os.path.join(subdir, file)
            rel_image_path = os.path.relpath(full_image_path, website_folder)
            # read in the images
            image = Image.open(full_image_path).convert("RGB")
            cityscapes_images.append(np.array(image))
            
            # path of corresponding label (color form)
            subfolder = file.split("_")[0] # the first word in the file corresponds to subfolder name
            parsed_name_parts = file.split("_")[:3]
            label_name = "_".join(parsed_name_parts)+"_gtFine_color.png"
#             label_name = label_name + 
            full_label_path = os.path.join(root_label_folder,subfolder, label_name)
            rel_label_path = os.path.relpath(full_label_path, website_folder)
#             label_class_path = "D:\\zsh\\graduation\\DAFormer\\data\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_labelIds.png"
            label_class_name = "_".join(parsed_name_parts)+"_gtFine_labelIds.png"
            label_class_path = os.path.join(root_label_folder,subfolder, label_class_name)
            label = Image.open(label_class_path).convert("L")
            cityscapes_labels.append(np.array(label))
            
            # path of predictions
            pred_name = file # same name as the input
            full_pred_path = os.path.join(root_predictions_folder,subfolder, pred_name)
            rel_pred_path = os.path.relpath(full_pred_path, website_folder)
            prediction = Image.open(full_pred_path).convert("RGB")# load predictions
            cityscapes_predictions.append(np.array(prediction))
            
            # add the paths
            cityscapes_full_paths.append((full_image_path,full_label_path))
            cityscapes_initial_info.append((file,rel_image_path,rel_label_path,rel_pred_path,"Cityscapes"))

# Print the results
print(cityscapes_full_paths[0])
print(cityscapes_initial_info[0]) # len 500

('C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\leftImg8bit\\val\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_color.png')
('frankfurt_000000_000294_leftImg8bit.png', 'dataset\\domain_adaptation\\Cityscapes\\leftImg8bit\\val\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'dataset\\domain_adaptation\\Cityscapes\\gtFine\\val\\frankfurt\\frankfurt_000000_000294_gtFine_color.png', 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds\\frankfurt\\frankfurt_000000_000294_leftImg8bit.png', 'Cityscapes')


In [12]:
import imageio.v2 as imageio

root_image_folder = synthia_image_folder
root_label_folder = synthia_label_image_folder
root_class_folder = synthia_label_class_folder
root_predictions_folder = synthia_predictions_folder

synthia_names = os.listdir(synthia_image_folder)
synthia_full_paths = []
synthia_images =[]
synthia_labels = []
synthia_predictions = []
synthia_initial_info = []

for file in synthia_names:
    # path of image
    full_image_path = os.path.join(root_image_folder,file)
    rel_image_path = os.path.relpath(full_image_path, website_folder)
    image = Image.open(full_image_path).convert("RGB")
    synthia_images.append(np.array(image))
    
    # path of corresponding label
    full_label_path = os.path.join(root_label_folder,file)
    rel_label_path = os.path.relpath(full_label_path, website_folder)

    # parsed_name = file.split(".")[0]
    # label_class_name = parsed_name+"_labelTrainIds.png"
    # label_class_path = os.path.join(root_class_folder,label_class_name)
    label_class_path = os.path.join(root_class_folder,file)
    # label_image = cv2.imread(label_class_path, cv2.IMREAD_UNCHANGED)
    label_image = imageio.imread(label_class_path,format="PNG-FI")
    label = np.asarray(label_image)[:,:,0]
    
    synthia_labels.append(np.array(label))
    
    # path of corresponding predictions
    full_pred_path = os.path.join(root_predictions_folder,file)
    rel_pred_path = os.path.relpath(full_pred_path, website_folder)
    prediction = Image.open(full_pred_path).convert("RGB")# load predictions
    synthia_predictions.append(np.array(prediction))
    
    synthia_full_paths.append((full_image_path,full_label_path))
    synthia_initial_info.append((file,rel_image_path,rel_label_path,rel_pred_path,"Synthia"))
# Print the results
print(synthia_full_paths[0])
print(synthia_initial_info[0])

('C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0000022.png', 'C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\GT\\COLOR\\0000022.png')
('0000022.png', 'dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0000022.png', 'dataset\\domain_adaptation\\Synthia\\GT\\COLOR\\0000022.png', 'dataset\\domain_adaptation\\Synthia\\predictions_500\\predictions\\0000022.png', 'Synthia')


# Labels to classes

In [14]:
# Cityscpaes
cityscapes_labels_arr = np.array(cityscapes_labels)
print(cityscapes_labels_arr.shape)

(500, 1024, 2048)


In [57]:
cityscapes_labels_arr[0]

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]], dtype=uint8)

In [132]:
new_class_names = ["unlabelled","road","sidewalk","vegetation","sky","car"]
# mapping the classes to 6 classes
# when using color to class, 0-3 is ignored
category_map = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 1,
    8: 2,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 3,
    22: 0,
    23: 4,
    24: 0,
    25: 0,
    26: 5,
    27: 0,
    28: 0,
    29: 0,
    30: 0,
    31: 0,
    32: 0,
    33: 0
}

In [16]:
# transform a colorful mask image to mask with classes
def map_to_6_classes(label,categorical_map):
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            idx = label[row, col]
            # d = cdist(np.array([a]),np.array(cityscapes_colors))
            # idx = np.argmin(d)
            new_idx = categorical_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [23]:
new_class_names = ["unlabelled","road","sidewalk","vegetation","sky","car"]

category_map_synthia = {
    0: 0,
    1: 4,
    2: 0,
    3: 1,
    4: 2,
    5: 0,
    6: 3,
    7: 0,
    8: 5,
    9: 0,
    10: 0,
    11: 0,
    12: 0,
    13: 0,
    14: 0,
    15: 0,
    16: 0,
    17: 0,
    18: 0,
    19: 0,
    20: 0,
    21: 3,
    22: 0,
    255:0,
}

In [21]:
start_time = time()
cityscapes_labels_6_classes = []
for i,label in enumerate(cityscapes_labels_arr):
    if i%50==0:
        print(i)
    new_label = map_to_6_classes(label,category_map)
    cityscapes_labels_6_classes.append(new_label)
end_time = time()
print(end_time-start_time) # 885 s = 15 min

0
50
100
150
200
250
300
350
400
450
946.3133473396301


In [24]:
synthia_labels_arr = np.array(synthia_labels)
print(synthia_labels_arr.shape)

start_time = time()
synthia_labels_6_classes = []
for i,label in enumerate(synthia_labels_arr):
    if i%50==0:
        print(i)
    new_label = map_to_6_classes(label,category_map_synthia)
    synthia_labels_6_classes.append(new_label)
end_time = time()
print(end_time-start_time) # 

print(synthia_labels_6_classes[0])

(500, 760, 1280)
0
50
100
150
200
250
300
350
400
450
443.0694227218628
[[0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 ...
 [1 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]
 [1 1 1 ... 2 2 2]]


# Test out color encoding with sample images

In [37]:
# randomly choose 10 images for each datasec
cityscape_labels_sample = cityscapes_labels_6_classes[:5]

In [45]:
np.unique(cityscape_labels_sample[0])

array([0, 1, 2, 3, 4, 5])

In [38]:
cityscape_labels_sample[0].shape

(1024, 2048)

In [56]:
print(cityscape_labels_sample[0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [153]:
# class to color: for viewing labels
selected_light_colors = {
    "others": [222, 203, 228],
    "road": [254, 217, 166],
    "sidewalk": [255, 255, 204],
    "vegetation": [229, 216, 189],
    "sky": [253, 218, 236],
    "car": [251, 180, 174]
}
class_names = ["others","road","sidewalk","vegetation","sky","car"]

def class_to_color(labels,dataset):
    if dataset =="Synthia":
        label_colors = np.zeros((760,1280,3))
    else:
        label_colors = np.zeros((1024,2048,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            # print(pixel, type(pixel))
            class_name = class_names[pixel]
            label_colors[i,j] = selected_light_colors[class_name]
    
    return label_colors.astype(np.uint8)  # make each pixel value an integer to visualize it better

In [47]:
from IPython.display import display

In [58]:
import matplotlib.pyplot as plt

In [80]:
cityscapes_initial_info[i][0]

'frankfurt_000000_000294_leftImg8bit.png'

In [84]:
website_folder = "C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image"

In [160]:
new_labels_path ="C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\New_labels"
new_label_paths_cityscapes = []

for i, label in enumerate(cityscapes_labels_6_classes):
    name = cityscapes_initial_info[i][0]
    new_label_arr = class_to_color(label,"Cityscapes")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    # Display the image
    new_saved_path = os.path.join(new_labels_path,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_label_paths_cityscapes.append(new_rel_path)
    break

In [161]:
new_labels_path_synthia ="C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\new labels"
new_label_paths_synthia = []

for i, label in enumerate(synthia_labels_6_classes):
    name = synthia_initial_info[i][0]
    new_label_arr = class_to_color(label,dataset="Synthia")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    # Display the image
    new_saved_path = os.path.join(new_labels_path_synthia,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_label_paths_synthia.append(new_rel_path)
    break

In [78]:
print(new_label_arr[0])

[[222 203 228]
 [222 203 228]
 [222 203 228]
 ...
 [222 203 228]
 [222 203 228]
 [222 203 228]]


In [75]:
new_label_arr.dtype

dtype('int32')

In [104]:
np.array(new_label_paths_cityscapes).shape

(500,)

In [113]:
new_label_paths_combined = np.hstack((np.array(new_label_paths_cityscapes),np.array(new_label_paths_synthia)))

In [106]:
new_label_paths_combined.shape

(1000,)

# Prediction

In [123]:
cityscapes_colors = [[  0,   0,   0], # this include five classes, but only one could be matched
         [111, 74,  0], # dynamic
         [81,0,81], # ground
        [128, 64, 128], # road
        [244, 35, 232], # sidewalk
         [250,170,160], # parking
         [230,150,140], # railtrack
        [70, 70, 70], # building
        [102, 102, 156],# wall
        [190, 153, 153],# fence
         [180,165,180], # gard rail
         [150,100,100], # bridge
         [150,120,90], # tunnel
        [153, 153, 153], # pole
         [153, 153, 153],# polegroup
        [250, 170, 30], # traffic light
        [220, 220, 0],  # traffic sign
        [107, 142, 35], # vegetation
        [152, 251, 152], # terrain
        [70, 130, 180], # sky
        [220, 20, 60], # person
        [255, 0, 0], # rider
        [0, 0, 142], # car
        [0, 0, 70], # truck
        [0, 60, 100], # bus
         [0,0,90], # caravan
         [ 0, 0,110], # trailor
        [0, 80, 100], # train
        [0, 0, 230], # motorcycle
        [119, 11, 32], # bicycle
    ]

In [126]:
def find_index_2d(array, subarray):
    for index, row in enumerate(array):
        if np.array_equal(row, subarray):
            return index
    return -1  # return -1 if the subarray is not found

def color_to_class(label,categorical_map,colors,dataset_name):
    # create new empty mask
    mask = np.zeros(shape=(label.shape[0], label.shape[1]), dtype = np.int32)
    # iterate through two dimensions
    for row in range(label.shape[0]):
        for col in range(label.shape[1]):
            a = label[row, col,:]
            # distance between this pixel and the original pixel
            # d = cdist(np.array([a]),np.array(synthia_colors))
            # idx = np.argmin(d)
            idx = find_index_2d(colors,a)
            if dataset_name == "Cityscapes":
                new_idx = categorical_map[idx+4]
            elif dataset_name == "Synthia":
                new_idx = categorical_map[idx]
            mask[row, col] = new_idx
    mask = np.reshape(mask, (mask.shape[0], mask.shape[1]))
    return mask

In [171]:
new_predictions_path ="C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color"
new_pred_paths_cityscapes = []

for i, prediction in enumerate(cityscapes_predictions):
    start_time = time()
    name = cityscapes_initial_info[i][0]
    # read in the color map 
    prediction = cityscapes_predictions[i]
    prediction_mask = color_to_class(prediction,category_map,cityscapes_colors,"Cityscapes")

    # transform the class mask to new color
    new_label_arr = class_to_color(prediction_mask,"Cityscapes")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')

    # save the image
    new_saved_path = os.path.join(new_predictions_path,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_pred_paths_cityscapes.append(new_rel_path)
    end_time = time()
    print(end_time-start_time)

39.643006563186646
42.31039047241211
39.014639139175415
50.26847743988037
39.77276849746704
37.011112689971924
45.254631757736206
45.40623998641968
47.93708825111389
43.653191804885864
46.948304891586304
48.89936876296997
47.99047660827637
43.497753381729126
50.750311851501465
42.8516800403595
44.07651472091675
38.2098069190979
45.11324334144592
43.12483835220337
44.64757966995239
48.2532479763031
43.401482582092285
36.63006901741028
42.026633501052856
47.970184087753296
49.39930462837219
38.71468424797058
44.91210198402405
49.96858024597168
43.20796036720276
35.84303164482117
42.291919469833374
41.03705549240112
44.15834927558899
42.37404680252075
42.03524136543274
44.746577501297
49.35502910614014
46.64008402824402
47.1719810962677
43.13831567764282
36.38710021972656
48.57198619842529
41.59088635444641
43.341410398483276
32.663548946380615
40.83671593666077
41.419865131378174
42.34655022621155
38.676209449768066
42.757083892822266
41.13624167442322
43.23944926261902
40.78040051460266

In [172]:
new_pred_paths_cityscapes[:3]

['dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000294_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000576_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_001016_leftImg8bit.png']

In [173]:
new_predictions_path_synthia ="C:\\Users\\20211016\\Graduation_project\\Graduation-project-domain-shift-image-2-image\\dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color"
new_pred_paths_synthia = []

for i, label in enumerate(synthia_predictions):
    start_time = time()
    name = synthia_initial_info[i][0]
    prediction = synthia_predictions[i]
    prediction_mask = color_to_class(prediction,category_map,cityscapes_colors,"Cityscapes")
    print(np.unique(prediction_mask))

    # transform the class mask to new color
    new_label_arr = class_to_color(prediction_mask,dataset="Synthia")
    new_label_img = Image.fromarray(new_label_arr, 'RGB')
    
    # save the image
    new_saved_path = os.path.join(new_predictions_path_synthia,name)
    new_label_img.save(new_saved_path)

    new_rel_path = os.path.relpath(new_saved_path, website_folder)
    new_pred_paths_synthia.append(new_rel_path)
    end_time = time()
    print(end_time-start_time)

[0 1 2 3 4 5]
18.679197549819946
[0 1 2 3 4 5]
22.735677480697632
[0 1 2 3 4 5]
24.400793075561523
[0 1 2 3 4 5]
21.174221992492676
[0 1 2 3 4 5]
28.53082275390625
[0 1 2 3 4 5]
23.0206036567688
[0 1 2 3 4 5]
25.619452953338623
[0 1 2 3 4 5]
28.452571153640747
[0 1 2 3 4 5]
27.515859365463257
[0 1 2 3 4 5]
19.759517669677734
[0 1 2 3 4 5]
20.281097412109375
[0 1 2 3 4 5]
26.14364767074585
[0 1 2 3 4 5]
21.886521816253662
[0 1 2 3 4 5]
26.76987886428833
[0 1 2 3 4 5]
23.052147388458252
[0 1 2 3 4 5]
25.585317611694336
[0 1 2 3 4 5]
21.475255250930786
[0 1 2 3 4 5]
23.912446975708008
[0 1 2 3 4 5]
26.818774461746216
[0 1 2 3 4 5]
27.241203784942627
[0 1 2 3 4 5]
26.24071192741394
[0 1 2 3 4 5]
24.28468346595764
[0 1 2 3 4 5]
20.038824796676636
[0 1 2 3 4 5]
19.521721839904785
[0 1 2 3 4 5]
30.167400598526
[0 1 2 3 4 5]
28.938631772994995
[0 1 2 3 4 5]
26.195747137069702
[0 1 2 3 4 5]
22.0950825214386
[0 1 2 3 4 5]
30.365943908691406
[0 1 2 3 4 5]
27.60047173500061
[0 1 2 3 4 5]
30.023552

In [174]:
new_pred_paths_synthia[:3]

['dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000022.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000026.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000037.png']

In [175]:
new_pred_paths_cityscapes.extend(new_pred_paths_synthia) # now new pred_paths include both cityscapes and synthia prediction paths

In [179]:
new_pred_paths_cityscapes[495:505]

['dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\munster_000169_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\munster_000170_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\munster_000171_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\munster_000172_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\munster_000173_000019_leftImg8bit.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000022.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000026.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000037.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000042.png',
 'dataset\\domain_adaptation\\Synthia\\predictions_500\\preds_new_color\\0000052.png']

In [176]:
len(combined_pred_path)

TypeError: object of type 'NoneType' has no len()

# combine the two paths and add to current df

In [158]:
new_label_paths_synthia[-1]

'dataset\\domain_adaptation\\Synthia\\new labels\\0000022.png'

In [137]:
print(end_time-start_time)

39.345325231552124


# Read the original data in

In [99]:
df_old = pd.read_csv("discreted_data_v14.csv",index_col="id")
df_old.head()

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\gtFine\va...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340


In [114]:
df_old.drop(columns = ["label_path"])

,name,dataset,image_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_ada

In [115]:
df_old["label_path"] = new_label_paths_combined.tolist()

In [116]:
df_old

,name,dataset,image_path,label_path,prediction_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Cityscapes\outputs\p...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Synthia\predictions_...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
9

In [138]:
df_old

Index(['name', 'dataset', 'image_path', 'label_path', 'prediction_path',
       'similar_image_paths', 'similar_IoU_score', 'other_ratio', 'road_ratio',
       'sidewalk_ratio', 'vegetation_ratio', 'sky_ratio', 'car_ratio',
       'simple_tsne_1', 'simple_tsne_2', 'meaningful_tsne_1',
       'meaningful_tsne_2', 'pca_1', 'pca_2', 'umap_1', 'umap_2',
       'bottleneck_tsne_embedding', 'bottleneck_pca_embedding', 'overall_iou',
       'other_iou', 'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou',
       'car_iou', 'similar_image_paths_others', 'similar_IoU_score_others',
       'similar_image_paths_road', 'similar_IoU_score_road',
       'similar_image_paths_sidewalk', 'similar_IoU_score_sidewalk',
       'similar_image_paths_vegetation', 'similar_IoU_score_vegetation',
       'similar_image_paths_sky', 'similar_IoU_score_sky',
       'similar_image_paths_car', 'similar_IoU_score_car'],
      dtype='object')

In [139]:
df_old["similar_image_paths_vegetation"][0]

'dataset\\domain_adaptation\\Synthia\\RGB_subset_500\\0003913.png'

In [180]:
df_old.drop(columns = ["prediction_path"])

,name,dataset,image_path,label_path,similar_image_paths,similar_IoU_score,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,...,similar_image_paths_road,similar_IoU_score_road,similar_image_paths_sidewalk,similar_IoU_score_sidewalk,similar_image_paths_vegetation,similar_IoU_score_vegetation,similar_image_paths_sky,similar_IoU_score_sky,similar_image_paths_car,similar_IoU_score_car
id,,,,,,,,,,,,,,,,,,,,,
0,frankfurt_000000_000294_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.660623,0.528985,0.297838,0.080459,0.020084,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.669665,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.445261,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.526673,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.516886,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.220297
1,frankfurt_000000_000576_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.581997,0.384708,0.345542,0.019122,0.085221,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.625078,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.192217,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.338053,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.546513,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.328769
2,frankfurt_000000_001016_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.663334,0.505177,0.336215,0.023324,0.011662,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.603962,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.205325,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.126580,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.000000,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.309487
3,frankfurt_000000_001236_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.484486,0.179743,0.381479,0.009730,0.256977,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.685399,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.090810,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.425622,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.459195,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.343273
4,frankfurt_000000_001751_leftImg8bit.png,Cityscapes,dataset\domain_adaptation\Cityscapes\leftImg8b...,dataset\domain_adaptation\Cityscapes\New_label...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.659385,0.405927,0.379869,0.024012,0.052703,...,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.682732,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.358655,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.569467,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.642079,dataset\domain_adaptation\Synthia\RGB_subset_5...,0.213340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0009354.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.472945,0.340820,0.336541,0.197154,0.094768,...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.414943,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.326770,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.252685,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.000346,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.053920
996,0009359.png,Synthia,dataset\domain_adaptation\Synthia\RGB_subset_5...,dataset\domain_adaptation\Synthia\new labels\0...,dataset\domain_adaptation\Cityscapes\leftImg8b...,0.380413,0.284762,0.331777,0.245871,0.107303,...,dataset\domain_adaptati

In [181]:
df_old["prediction_path"]=new_pred_paths_cityscapes

In [183]:
df_old["prediction_path"].iloc[0]

'dataset\\domain_adaptation\\Cityscapes\\outputs\\preds_new_color\\frankfurt_000000_000294_leftImg8bit.png'

# Still need to change
* prediction_path (need hours of running at first)

In [122]:
df_old["label_path"].iloc[579]

'dataset\\domain_adaptation\\Synthia\\new labels\\0001628.png'

In [184]:
df_old.to_csv("discreted_data_v16.csv",index = "id")